# ขั้นตอน 1: โหลดและเตรียมข้อมูล

In [ ]:
import pandas as pd

# โหลดข้อมูลสภาพอากาศจากหลายชีทในไฟล์ rain.xlsx
file_path = 'rain.xlsx'
xls = pd.ExcelFile(file_path)

df_list = []
for sheet in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet)
    df = df.rename(columns={'ปี': 'year', 'เดือน': 'month', 'วันที่': 'day'})

    # แปลงเป็นตัวเลข
    df['year'] = pd.to_numeric(df['year'], errors='coerce')
    df['month'] = pd.to_numeric(df['month'], errors='coerce')
    df['day'] = pd.to_numeric(df['day'], errors='coerce')
    df = df.dropna(subset=['year', 'month', 'day'])

    # สร้าง datetime column
    df['วัน'] = pd.to_datetime(df[['year', 'month', 'day']], errors='coerce')
    df = df.dropna(subset=['วัน'])

    # แปลงข้อมูลสภาพอากาศให้เป็นตัวเลข
    for col in ['อุณหภูมิ', 'ฝน', 'ชื้น']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # กรองเอาเฉพาะแถวที่มีข้อมูลครบ
    df = df[df[['อุณหภูมิ', 'ฝน', 'ชื้น']].notna().all(axis=1)]
    df_list.append(df)

# รวมข้อมูลสภาพอากาศทั้งหมด
weather = pd.concat(df_list, ignore_index=True).drop_duplicates().sort_values('วัน').reset_index(drop=True)

# บันทึกข้อมูลสภาพอากาศที่ clean แล้ว
weather.to_excel('rain_all_years_sorted3.xlsx', index=False)
print("✅ บันทึก rain_all_years_sorted3.xlsx เรียบร้อยแล้ว")


# ขั้นตอน 2: โหลดและเตรียมข้อมูลผู้ป่วย

In [ ]:
# โหลดข้อมูลผู้ป่วยจากหลายชีทในไฟล์ DHFcc12.xlsx
xls = pd.ExcelFile('DHFcc12.xlsx')
df_list = []

for sheet in xls.sheet_names:
    if sheet == "Table1":  # ข้ามชีท Table1
        continue

    df = pd.read_excel(xls, sheet_name=sheet)
    df.columns = df.columns.str.strip()

    if 'วันเริ่มป่วย' in df.columns:
        df['วันเริ่มป่วย'] = pd.to_datetime(df['วันเริ่มป่วย'], errors='coerce')

    for col in ['อายุ(ปี)', 'อายุ(เดือน)']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    df = df.dropna(subset=['วันเริ่มป่วย'])
    df = df.dropna(how='all')

    df_list.append(df)

patients = pd.concat(df_list, ignore_index=True).drop_duplicates()
print(f"✅ รวมข้อมูลผู้ป่วยสำเร็จ: {patients.shape[0]} แถว, {patients.shape[1]} คอลัมน์")


# ขั้นตอน 3: คำนวณ rolling average ของสภาพอากาศย้อนหลัง 15 วัน สำหรับแต่ละวันเริ่มป่วย

In [ ]:
# โหลดข้อมูลสภาพอากาศที่จัดการแล้ว (เพื่อความแน่นอน)
weather = pd.read_excel("rain_all_years_sorted3.xlsx")
weather['วัน'] = pd.to_datetime(weather['วัน'])

# ฟังก์ชันคำนวณ rolling average ย้อนหลัง 15 วัน
def get_rolling_avg(date, n_days=15):
    start_date = date - pd.Timedelta(days=n_days)
    mask = (weather['วัน'] >= start_date) & (weather['วัน'] < date)
    subset = weather.loc[mask]
    if subset.empty:
        return pd.Series([None, None, None])
    return pd.Series([
        subset['อุณหภูมิ'].mean(),
        subset['ฝน'].mean(),
        subset['ชื้น'].mean()
    ])

# คำนวณ rolling average สำหรับผู้ป่วยแต่ละราย
patients[['temp_15d_avg', 'rain_15d_avg', 'humid_15d_avg']] = patients['วันเริ่มป่วย'].apply(get_rolling_avg)
print("✅ คำนวณ rolling average เสร็จ")


# ขั้นตอน 4: สร้างฟีเจอร์วันที่ เช่น ปี, เดือน, ไตรมาส, ฤดูกาล

In [ ]:
patients['ปี'] = patients['วันเริ่มป่วย'].dt.year
patients['เดือน'] = patients['วันเริ่มป่วย'].dt.month
patients['ไตรมาส'] = patients['วันเริ่มป่วย'].dt.quarter

def assign_season(row):
    month = row['เดือน']
    day = row['วันเริ่มป่วย'].day
    if (month == 2 and day >= 15) or month in [3,4] or (month == 5 and day < 15):
        return 'Summer'
    elif (month == 5 and day >= 15) or month in [6,7,8,9] or (month == 10 and day < 15):
        return 'Rainy'
    else:
        return 'Winter'

patients['ฤดูกาล'] = patients.apply(assign_season, axis=1)


# ขั้นตอน 5: รวมข้อมูลรายวันและคำนวณจำนวนผู้ป่วยรายวัน

In [ ]:
# สร้าง column วันแบบ date (ไม่มีเวลา)
patients['date'] = patients['วันเริ่มป่วย'].dt.date

# นับจำนวนผู้ป่วยแต่ละวัน (cases)
daily_counts = patients.groupby('date').size().reset_index(name='cases')

# คำนวณค่าเฉลี่ยและโหมดของตัวแปรอื่น ๆ ต่อวัน
daily_weather = patients.groupby('date').agg({
    'temp_15d_avg': 'mean',
    'rain_15d_avg': 'mean',
    'humid_15d_avg': 'mean',
    'อายุ(ปี)': 'mean',
    'เพศ': lambda x: x.mode()[0] if not x.mode().empty else None,
    'อาชีพ': lambda x: x.mode()[0] if not x.mode().empty else None,
    'ตำบล': lambda x: x.mode()[0] if not x.mode().empty else None,
    'อำเภอ': lambda x: x.mode()[0] if not x.mode().empty else None,
    'เดือน': lambda x: x.mode()[0] if not x.mode().empty else None,
    'ปี': lambda x: x.mode()[0] if not x.mode().empty else None,
    'ไตรมาส': lambda x: x.mode()[0] if not x.mode().empty else None,
    'ฤดูกาล': lambda x: x.mode()[0] if not x.mode().empty else None
}).reset_index()

# รวมข้อมูลผู้ป่วยรายวันกับข้อมูลสภาพอากาศ
df_final = pd.merge(daily_counts, daily_weather, on='date', how='left')
print("✅ รวมข้อมูล df_final สำเร็จ:", df_final.shape)


# ขั้นตอน 6: จัดการ missing values และบันทึกข้อมูล

In [ ]:
missing_before = df_final.isnull().sum().sum()
print(f"⚠️ พบ missing values: {missing_before} ช่อง")

df_final_clean = df_final.dropna()
print("✅ ลบ missing rows แล้ว → ขนาดข้อมูล:", df_final_clean.shape)

# บันทึกไฟล์ clean data
df_final_clean.to_csv("df_final_clean.csv", index=False)
df_final_clean.to_excel("df_final_clean.xlsx", index=False)
print("📁 บันทึกไฟล์ df_final_clean.csv และ df_final_clean.xlsx เรียบร้อย")


# ขั้นตอน 7: เปลี่ยนชื่อคอลัมน์เป็นภาษาอังกฤษ เพื่อใช้งานในโมเดล

In [ ]:
df_final_clean = pd.read_excel("/content/df_final_clean.xlsx")

In [ ]:
print(df_final_clean.isnull().sum())


In [ ]:
df_final_clean = df_final_clean.drop_duplicates() #ตรวจสอบข้อมูลซ้ำ (Duplicate Records)

In [ ]:
print(df_final_clean.dtypes) #ตรวจสอบประเภทข้อมูล (Data Types)


In [ ]:
df_clean = df_final_clean.rename(columns={
    'เพศ': 'gender',
    'อาชีพ': 'occupation',
    'ตำบล': 'subdistrict',
    'อำเภอ': 'district',
    'เดือน': 'month',
    'ไตรมาส': 'quarter',
    'อายุ(ปี)': 'age',
    'ฤดูกาล': 'season',
})


In [ ]:
print(df_clean.head())

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# โหลดข้อมูลจากไฟล์
df_final_clean = pd.read_excel("/content/df_final_clean.xlsx")

# ตัวอย่างการสร้างกราฟแท่ง (Bar Chart) แสดงการกระจายของ 'cases' ตาม 'season'
plt.figure(figsize=(10, 6))
df_final_clean.groupby('ฤดูกาล')['cases'].sum().plot(kind='bar', color='skyblue')
plt.title('Total Cases by Season')
plt.xlabel('Season')
plt.ylabel('Total Cases')
plt.xticks(rotation=45)
plt.show()

# ตัวอย่างการสร้างกราฟกระจาย (Scatter Plot) แสดงความสัมพันธ์ระหว่าง 'age' และ 'cases'
plt.figure(figsize=(10, 6))
plt.scatter(df_final_clean['อายุ(ปี)'], df_final_clean['cases'], color='orange')
plt.title('Scatter Plot of Age vs Cases')
plt.xlabel('Age')
plt.ylabel('Cases')
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# แปลงคอลัมน์ 'date' เป็น datetime
df_clean['date'] = pd.to_datetime(df_clean['date'], errors='coerce')

# กรองข้อมูลในช่วงปี 2017-2024
df_clean = df_clean[df_clean['date'].dt.year.between(2017, 2024)]

# สร้างคอลัมน์ 'month' และ 'year' เพื่อให้แยกกราฟตามเดือน
df_clean['month'] = df_clean['date'].dt.month
df_clean['year'] = df_clean['date'].dt.year

# คำนวณค่าเฉลี่ยตามปีและเดือน
avg_data = df_clean.groupby(['year', 'month']).agg({
    'temp_15d_avg': 'mean',
    'rain_15d_avg': 'mean',
    'humid_15d_avg': 'mean'
}).reset_index()

# ตั้งค่าการแสดงผล 3 กราฟ (temp_15d_avg, rain_15d_avg, humid_15d_avg) แยกตามเดือน
fig, axs = plt.subplots(3, 1, figsize=(12, 18), sharex=True)

# แสดงกราฟแค่เดือนที่จำเป็น (ทุกๆ 6 เดือน)
months_to_display = [1, 6, 12]
avg_data_filtered = avg_data[avg_data['month'].isin(months_to_display)]

# กราฟแสดงอุณหภูมิ (Temperature)
axs[0].plot(avg_data_filtered['year'].astype(str) + '-' + avg_data_filtered['month'].astype(str), avg_data_filtered['temp_15d_avg'], color='red', label='Temperature')
axs[0].set_title('Average Temperature (°C) from 2017 to 2024')
axs[0].set_ylabel('Temperature (°C)')
axs[0].legend()

# กราฟแสดงฝน (Rainfall)
axs[1].plot(avg_data_filtered['year'].astype(str) + '-' + avg_data_filtered['month'].astype(str), avg_data_filtered['rain_15d_avg'], color='green', label='Rainfall')
axs[1].set_title('Average Rainfall (mm) from 2017 to 2024')
axs[1].set_ylabel('Rainfall (mm)')
axs[1].legend()

# กราฟแสดงความชื้น (Humidity)
axs[2].plot(avg_data_filtered['year'].astype(str) + '-' + avg_data_filtered['month'].astype(str), avg_data_filtered['humid_15d_avg'], color='blue', label='Humidity')
axs[2].set_title('Average Humidity (%) from 2017 to 2024')
axs[2].set_ylabel('Humidity (%)')
axs[2].legend()

# ตั้งชื่อกราฟ
plt.xlabel('Date (Year-Month)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# อ่านข้อมูล
df = pd.read_csv('df_final_clean.csv')  # ใส่ path ของไฟล์ที่คุณใช้

# แปลงคอลัมน์ 'date' เป็น datetime
df['date'] = pd.to_datetime(df['date'])

# สร้างคอลัมน์ 'year' และ 'season' สำหรับกราฟรายปีและฤดูกาล
df['year'] = df['date'].dt.year  # เพิ่มคอลัมน์ 'year' สำหรับการกรุ๊ปข้อมูลรายปี
df['season'] = df['date'].dt.month % 12 // 3 + 1  # ฤดูกาล: 1=ฤดูร้อน, 2=ฤดูฝน, 3=ฤดูหนาว

# คำนวณค่าเฉลี่ยประจำปี
df_yearly = df.groupby('year').agg({
    'humid_15d_avg': 'mean',
    'rain_15d_avg': 'mean',
    'temp_15d_avg': 'mean'
}).reset_index()

# สร้างกราฟ
fig, ax = plt.subplots(3, 1, figsize=(10, 12))

# กราฟแสดงอุณหภูมิ
ax[0].plot(df_yearly['year'], df_yearly['temp_15d_avg'], label='Temperature', color='red')
ax[0].set_title('Average Temperature from 2017 to 2024')
ax[0].set_xlabel('Year')
ax[0].set_ylabel('Temperature (°C)')

# กราฟแสดงฝน
ax[1].plot(df_yearly['year'], df_yearly['rain_15d_avg'], label='Rain', color='blue')
ax[1].set_title('Average Rainfall from 2017 to 2024')
ax[1].set_xlabel('Year')
ax[1].set_ylabel('Rainfall (mm)')

# กราฟแสดงความชื้น
ax[2].plot(df_yearly['year'], df_yearly['humid_15d_avg'], label='Humidity', color='green')
ax[2].set_title('Average Humidity from 2017 to 2024')
ax[2].set_xlabel('Year')
ax[2].set_ylabel('Humidity (%)')

# แสดงกราฟ
plt.tight_layout()
plt.legend()
plt.show()


In [ ]:
print(df.columns)


In [ ]:
import matplotlib.pyplot as plt
from matplotlib import font_manager

# กำหนดฟอนต์ที่ใช้
font_path = "/content/THSarabunNew.ttf"  # แก้ไขให้ตรงกับตำแหน่งฟอนต์ที่ติดตั้ง
font_prop = font_manager.FontProperties(fname=font_path)

# คำนวณจำนวนผู้ป่วยทั้งหมดในแต่ละตำบล
top_subdistricts = df_clean.groupby('subdistrict')['cases'].sum().sort_values(ascending=False).head(20)

# สร้างกราฟ
plt.figure(figsize=(12, 8))
top_subdistricts.plot(kind='barh', color='lightblue')

# ตั้งชื่อกราฟและแกน
plt.xlabel('Total Cases', fontproperties=font_prop)
plt.ylabel('Subdistrict', fontproperties=font_prop)
plt.title('Top 20 Subdistricts by Total Cases', fontproperties=font_prop)

# หมุนชื่อให้สามารถอ่านได้ง่ายขึ้น
plt.xticks(rotation=45, fontproperties=font_prop)
plt.yticks(fontproperties=font_prop)

# เพิ่มข้อความแสดงจำนวนผู้ป่วย
for index, value in enumerate(top_subdistricts):
    plt.text(value, index, str(value), fontproperties=font_prop)  # แสดงค่าในแต่ละตำแหน่งของบาร์

plt.show()


In [ ]:
import matplotlib.pyplot as plt
from matplotlib import font_manager

# กำหนดฟอนต์ที่ใช้
font_path = "/content/THSarabunNew.ttf"  # แก้ไขให้ตรงกับตำแหน่งฟอนต์ที่ติดตั้ง
font_prop = font_manager.FontProperties(fname=font_path)

# คำนวณจำนวนผู้ป่วยทั้งหมดในแต่ละตำบล
top_subdistricts = df_clean.groupby('subdistrict')['cases'].sum().sort_values(ascending=True).head(20)  # เลือก 20 ตำบลที่มีจำนวนผู้ป่วยน้อยที่สุด

# สร้างกราฟ
plt.figure(figsize=(12, 8))
top_subdistricts.plot(kind='barh', color='lightblue')

# ตั้งชื่อกราฟและแกน
plt.xlabel('Total Cases', fontproperties=font_prop)
plt.ylabel('Subdistrict', fontproperties=font_prop)
plt.title('Top 20 Subdistricts by Total Cases (Lowest)', fontproperties=font_prop)

# หมุนชื่อให้สามารถอ่านได้ง่ายขึ้น
plt.xticks(rotation=45, fontproperties=font_prop)
plt.yticks(fontproperties=font_prop)

# เพิ่มข้อความแสดงจำนวนผู้ป่วย
for index, value in enumerate(top_subdistricts):
    plt.text(value, index, str(value), fontproperties=font_prop)  # แสดงค่าในแต่ละตำแหน่งของบาร์

plt.show()


In [ ]:
import matplotlib.pyplot as plt
from matplotlib import font_manager

# กำหนดฟอนต์ที่ใช้
font_path = "/content/THSarabunNew.ttf"  # แก้ไขให้ตรงกับตำแหน่งฟอนต์ที่ติดตั้ง
font_prop = font_manager.FontProperties(fname=font_path)

# คำนวณจำนวนผู้ป่วยทั้งหมดในแต่ละอาชีพ
top_occupation = df_clean.groupby('occupation')['cases'].sum().sort_values(ascending=False).head(20)

# สร้างกราฟ
plt.figure(figsize=(12, 10))  # เพิ่มขนาดกราฟ
top_occupation.plot(kind='barh', color='lightcoral')  # ใช้ top_occupation แทน top_subdistricts

# ตั้งชื่อกราฟและแกน
plt.xlabel('Total Cases', fontproperties=font_prop)
plt.ylabel('Occupation', fontproperties=font_prop)  # เปลี่ยนชื่อแกน y
plt.title('Top 20 Occupation by Total Cases', fontproperties=font_prop)  # เปลี่ยนชื่อกราฟ

# หมุนชื่อให้สามารถอ่านได้ง่ายขึ้น
plt.xticks(rotation=45, fontproperties=font_prop)
plt.yticks(fontproperties=font_prop)

# เพิ่มข้อความแสดงจำนวนผู้ป่วย
for index, value in enumerate(top_occupation):  # ใช้ top_occupation แทน top_subdistricts
    plt.text(value, index, str(value), fontproperties=font_prop)  # แสดงค่าในแต่ละตำแหน่งของบาร์

plt.tight_layout()  # จัดการกับขนาดของกราฟให้สมบูรณ์
plt.show()


# ขั้นตอน 8: วิเคราะห์เชิงเดี่ยว (Univariate Analysis)

In [ ]:
# ตรวจสอบชื่อคอลัมน์อีกครั้ง
print(df_final_clean.columns)


In [ ]:
import pandas as pd
import scipy.stats as stats
from scipy.stats import spearmanr, pointbiserialr, kruskal

# เลือกตัวแปรที่ใช้
target = 'cases'
numeric_vars = ['temp_15d_avg', 'rain_15d_avg', 'humid_15d_avg', 'อายุ(ปี)']  # รวม 'อายุ(ปี)'
binary_cats = ['เพศ']
multi_cats = ['อาชีพ', 'ตำบล', 'อำเภอ', 'เดือน', 'ปี', 'ไตรมาส', 'ฤดูกาล']  # รวมตัวแปร 'เดือน', 'ปี', 'ไตรมาส', 'ฤดูกาล'

# แปลงเพศเป็นตัวเลข (ชาย=0, หญิง=1)
if 'เพศ' in df_final_clean.columns:
    df_final_clean['gender_code'] = df_final_clean['เพศ'].map({'ชาย': 0, 'หญิง': 1})

# ลบ missing values ที่จำเป็นสำหรับตัวแปรที่ใช้ในการคำนวณ
df_final_clean = df_final_clean.dropna(subset=['temp_15d_avg', 'rain_15d_avg', 'humid_15d_avg', 'gender_code', target])

# คำนวณค่า p-value
results = []

# 1) Spearman correlation กับตัวแปรเชิงปริมาณ
for var in numeric_vars:
    corr, p = spearmanr(df_final_clean[var], df_final_clean[target])
    results.append({
        'Variable': var,
        'Test': 'Spearman Correlation',
        'Stat': round(corr, 3),
        'P-Value': round(p, 4)
    })

# 2) Point-Biserial correlation กับตัวแปรไบนารี
for var in binary_cats:
    encoded_var = var + '_code'  # ชื่อคอลัมน์ที่แปลงจากเพศ
    if encoded_var in df_final_clean.columns and df_final_clean[encoded_var].nunique() == 2:
        corr, p = pointbiserialr(df_final_clean[encoded_var], df_final_clean[target])
        results.append({
            'Variable': var,
            'Test': 'Point-Biserial',
            'Stat': round(corr, 3),
            'P-Value': round(p, 4)
        })

# 3) Kruskal-Wallis test กับตัวแปรหลายกลุ่ม
for var in multi_cats:
    df_non_null = df_final_clean.dropna(subset=[var])
    groups = [group[target].dropna() for name, group in df_non_null.groupby(var)]
    if len(groups) > 1:
        h_stat, p = kruskal(*groups)
        results.append({
            'Variable': var,
            'Test': 'Kruskal-Wallis',
            'Stat': round(h_stat, 3),
            'P-Value': round(p, 4)
        })

# แสดงผล
results_df = pd.DataFrame(results).sort_values('P-Value')
print("\n📊 ผลวิเคราะห์เชิงเดี่ยว (เรียงตาม p-value):")
print(results_df)


In [ ]:
import pandas as pd
import scipy.stats as stats
from scipy.stats import spearmanr, pointbiserialr, kruskal

# เลือกตัวแปรที่ใช้
target = 'cases'
numeric_vars = ['temp_15d_avg', 'rain_15d_avg', 'humid_15d_avg', 'อายุ(ปี)']  # รวม 'อายุ(ปี)'
binary_cats = ['เพศ']
multi_cats = ['อาชีพ', 'ตำบล', 'อำเภอ', 'เดือน', 'ปี', 'ไตรมาส', 'ฤดูกาล']  # รวมตัวแปร 'เดือน', 'ปี', 'ไตรมาส', 'ฤดูกาล'

# แปลงเพศเป็นตัวเลข (ชาย=0, หญิง=1)
if 'เพศ' in df_final_clean.columns:
    df_final_clean['gender_code'] = df_final_clean['เพศ'].map({'ชาย': 0, 'หญิง': 1})

# ลบ missing values ที่จำเป็นสำหรับตัวแปรที่ใช้ในการคำนวณ
df_final_clean = df_final_clean.dropna(subset=['temp_15d_avg', 'rain_15d_avg', 'humid_15d_avg', 'gender_code', target])

# คำนวณค่า p-value
results = []

# 1) Spearman correlation กับตัวแปรเชิงปริมาณ
for var in numeric_vars:
    corr, p = spearmanr(df_final_clean[var], df_final_clean[target])
    results.append({
        'Variable': var,
        'Test': 'Spearman Correlation',
        'Stat': round(corr, 3),
        'P-Value': round(p, 4)
    })

# 2) Point-Biserial correlation กับตัวแปรไบนารี
for var in binary_cats:
    encoded_var = var + '_code'  # ชื่อคอลัมน์ที่แปลงจากเพศ
    if encoded_var in df_final_clean.columns and df_final_clean[encoded_var].nunique() == 2:
        corr, p = pointbiserialr(df_final_clean[encoded_var], df_final_clean[target])
        results.append({
            'Variable': var,
            'Test': 'Point-Biserial',
            'Stat': round(corr, 3),
            'P-Value': round(p, 4)
        })

# 3) Kruskal-Wallis test กับตัวแปรหลายกลุ่ม
for var in multi_cats:
    df_non_null = df_final_clean.dropna(subset=[var])
    groups = [group[target].dropna() for name, group in df_non_null.groupby(var)]
    if len(groups) > 1:
        h_stat, p = kruskal(*groups)
        results.append({
            'Variable': var,
            'Test': 'Kruskal-Wallis',
            'Stat': round(h_stat, 3),
            'P-Value': round(p, 4)
        })

# แปลงผลลัพธ์เป็น DataFrame
results_df = pd.DataFrame(results).sort_values('P-Value')

# กรองเฉพาะตัวแปรที่มี p-value < 0.05
significant_results_df = results_df[results_df['P-Value'] < 0.05]

# แสดงผล
print("\n📊 ผลวิเคราะห์เชิงเดี่ยว (ตัวแปรที่มี p-value < 0.05):")
print(significant_results_df)


# ขั้นตอน 9: วิเคราะห์เชิงพหุด้วย Negative Binomial Regression

In [ ]:
print(df_final_clean.columns)  # ตรวจสอบคอลัมน์ทั้งหมดใน DataFrame


In [ ]:
df_final_clean.rename(columns={
    'date': 'date',
    'cases': 'cases',
    'temp_15d_avg': 'temp_15d_avg',
    'rain_15d_avg': 'rain_15d_avg',
    'humid_15d_avg': 'humid_15d_avg',
    'อายุ(ปี)': 'age',
    'เพศ': 'gender',
    'อาชีพ': 'occupation',
    'ตำบล': 'subdistrict',
    'อำเภอ': 'district',
    'เดือน': 'month',
    'ปี': 'year',
    'ไตรมาส': 'quarter',
    'ฤดูกาล': 'season',
    'gender_code': 'gender_code'
}, inplace=True)


In [ ]:
import statsmodels.api as sm  # นำเข้า statsmodels


In [ ]:
# แปลงคอลัมน์ที่เป็น categorical ให้เป็น dtype 'category' (ตัวแปรหมวดหมู่)
df_final_clean['occupation'] = df_final_clean['occupation'].astype('category')
df_final_clean['subdistrict'] = df_final_clean['subdistrict'].astype('category')
df_final_clean['district'] = df_final_clean['district'].astype('category')
df_final_clean['season'] = df_final_clean['season'].astype('category')
df_final_clean['quarter'] = df_final_clean['quarter'].astype('category')

# ตรวจสอบข้อมูลใหม่
print(df_final_clean.dtypes)


In [ ]:
# ลบคอลัมน์ season ที่ซ้ำ
df_final_clean = df_final_clean.loc[:, ~df_final_clean.columns.duplicated()]

# สร้างโมเดล Negative Binomial
nb_model = smf.glm(
    formula=formula,
    data=df_final_clean,
    family=sm.families.NegativeBinomial()  # ใช้ sm.families.NegativeBinomial()
).fit()

# แสดงผลลัพธ์ของโมเดล
print(nb_model.summary())


In [ ]:
# สรุปเฉพาะตัวแปรที่มีนัยสำคัญ p-value < 0.05
summary_df = nb_model.summary2().tables[1]
significant_vars = summary_df[summary_df['P>|z|'] < 0.05]

print("\n📌 ตัวแปรที่มีนัยสำคัญ (p-value < 0.05):")
print(significant_vars[['Coef.', 'Std.Err.', 'z', 'P>|z|']])


In [ ]:
# ตรวจสอบ overdispersion
mean_cases = df_final_clean['cases'].mean()
var_cases = df_final_clean['cases'].var()

print(f"\nMean of cases: {mean_cases:.4f}")
print(f"Variance of cases: {var_cases:.4f}")

# ตรวจสอบว่าเกิด overdispersion หรือไม่
if var_cases > mean_cases:
    print("📈 มี overdispersion → เหมาะกับ Negative Binomial")
else:
    print("📉 ไม่มี overdispersion → อาจไม่เหมาะกับ Negative Binomial")


In [ ]:
from sklearn.preprocessing import StandardScaler

# การปรับข้อมูลให้เป็นมาตรฐาน (Standardization)
scaler = StandardScaler()

# ทำการ normalize ข้อมูลที่ใช้ในการพยากรณ์
df_final_clean[['temp_15d_avg', 'rain_15d_avg', 'humid_15d_avg']] = scaler.fit_transform(
    df_final_clean[['temp_15d_avg', 'rain_15d_avg', 'humid_15d_avg']])

# สร้างโมเดล Negative Binomial ใหม่
nb_model = smf.glm(
    formula=formula,
    data=df_final_clean,
    family=sm.families.NegativeBinomial()
).fit()

# ผลลัพธ์ใหม่
predictions = nb_model.predict(df_final_clean)


In [ ]:
# Use rolling average for smoothing
predictions_smooth = predictions.rolling(window=10).mean()
actual_cases_smooth = df_final_clean['cases'].rolling(window=10).mean()

# กราฟใหม่
plt.figure(figsize=(10,6))
plt.plot(predictions_smooth, label='Predicted Cases', color='blue')
plt.plot(actual_cases_smooth, label='Actual Cases', color='red', alpha=0.6)
plt.title("Smoothed Predicted vs Actual Cases")
plt.xlabel("Index")
plt.ylabel("Number of Cases")
plt.legend()
plt.show()


In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt

# เลือกเฉพาะตัวแปรที่มีนัยสำคัญ
significant_columns = ['month', 'rain_15d_avg', 'humid_15d_avg', 'subdistrict']

# แปลง categorical variables ให้เป็นตัวเลข
df_final_clean_encoded = df_final_clean[significant_columns].copy()

# ใช้ LabelEncoder สำหรับแปลงตัวแปรเชิงหมวดหมู่ให้เป็นตัวเลข
label_encoder = LabelEncoder()
df_final_clean_encoded['subdistrict'] = label_encoder.fit_transform(df_final_clean_encoded['subdistrict'])

# สร้าง X และ y
X = df_final_clean_encoded
y = df_final_clean['cases']

# แบ่งข้อมูลเป็นชุดฝึกและทดสอบ
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# สร้างโมเดล XGBoost และปรับพารามิเตอร์
model = xgb.XGBRegressor(
    objective='reg:squarederror',
    eval_metric='rmse',
    max_depth=5,        # ลดความลึกของต้นไม้
    learning_rate=0.05,  # ลดค่าความเร็วในการเรียนรู้
    n_estimators=500,   # เพิ่มจำนวนต้นไม้
    colsample_bytree=0.8,  # ตัวแปรที่ใช้ในการแบ่งแต่ละต้นไม้
    subsample=0.7,         # ขนาดของแต่ละชุดย่อยในการฝึก
    gamma=0.1           # การปรับให้โมเดลไม่ฟิตมากเกินไป
)

# ฝึกโมเดล
model.fit(X_train, y_train)

# ทำนายค่าบนชุดทดสอบ
y_pred = model.predict(X_test)

# คำนวณค่า MSE (Mean Squared Error)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)  # คำนวณค่ารากที่สองของ MSE
print(f"RMSE: {rmse}")

# คำนวณค่า R² (Coefficient of Determination)
r2 = r2_score(y_test, y_pred)
print(f"R²: {r2}")

# กราฟเปรียบเทียบผลทำนายและค่าจริง
plt.figure(figsize=(10,6))
plt.plot(y_test.values, label='Actual Cases', color='red')
plt.plot(y_pred, label='Predicted Cases', color='blue')
plt.title('Predicted vs Actual Cases')
plt.xlabel('Index')
plt.ylabel('Number of Cases')
plt.legend()
plt.show()

# คำนวณ Feature Importance
xgb.plot_importance(model, importance_type='weight', max_num_features=10, title="Feature Importance")
plt.show()


# XGBoost

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# เลือกเฉพาะตัวแปรที่มีนัยสำคัญ
significant_columns = ['rain_15d_avg', 'humid_15d_avg', 'month', 'subdistrict']

# แปลง categorical variables ให้เป็นตัวเลข
df_final_clean_encoded = df_final_clean[significant_columns].copy()

# ใช้ One-Hot Encoding สำหรับแปลงตัวแปร subdistrict เป็นค่าดิจิตอล
df_final_clean_encoded = pd.get_dummies(df_final_clean_encoded, columns=['subdistrict'], drop_first=True)

# ใช้ LabelEncoder สำหรับแปลงตัวแปรเดือน
label_encoder = LabelEncoder()
df_final_clean_encoded['month'] = label_encoder.fit_transform(df_final_clean_encoded['month'])

# สร้าง X และ y
X = df_final_clean_encoded
y = df_final_clean['cases']

# แบ่งข้อมูลเป็นชุดฝึกและทดสอบ
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# สร้างโมเดล XGBoost และปรับพารามิเตอร์
model = xgb.XGBRegressor(
    objective='reg:squarederror',
    eval_metric='rmse',
    max_depth=5,        # ลดความลึกของต้นไม้
    learning_rate=0.05,  # ลดค่าความเร็วในการเรียนรู้
    n_estimators=500,   # เพิ่มจำนวนต้นไม้
    colsample_bytree=0.8,  # ตัวแปรที่ใช้ในการแบ่งแต่ละต้นไม้
    subsample=0.7,         # ขนาดของแต่ละชุดย่อยในการฝึก
    gamma=0.1           # การปรับให้โมเดลไม่ฟิตมากเกินไป
)

# ฝึกโมเดล
model.fit(X_train, y_train)

# ทำนายค่าบนชุดทดสอบ
y_pred = model.predict(X_test)

# คำนวณค่า MSE (Mean Squared Error)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)  # คำนวณค่ารากที่สองของ MSE
print(f"RMSE: {rmse}")

# คำนวณค่า R² (Coefficient of Determination)
r2 = r2_score(y_test, y_pred)
print(f"R²: {r2}")

# กราฟเปรียบเทียบผลทำนายและค่าจริง
plt.figure(figsize=(10,6))
plt.plot(y_test.values, label='Actual Cases', color='red')
plt.plot(y_pred, label='Predicted Cases', color='blue')
plt.title('Predicted vs Actual Cases')
plt.xlabel('Index')
plt.ylabel('Number of Cases')
plt.legend()
plt.show()

# คำนวณ Feature Importance
xgb.plot_importance(model, importance_type='weight', max_num_features=10, title="Feature Importance")
plt.show()


# Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt

# เลือกเฉพาะตัวแปรที่มีนัยสำคัญ
significant_columns = ['rain_15d_avg', 'month', 'subdistrict', 'humid_15d_avg']

# แปลง categorical variables ให้เป็นตัวเลข
df_final_clean_encoded = df_final_clean[significant_columns].copy()

# ใช้ LabelEncoder สำหรับแปลงตัวแปรเชิงหมวดหมู่ให้เป็นตัวเลข
label_encoder = LabelEncoder()
df_final_clean_encoded['subdistrict'] = label_encoder.fit_transform(df_final_clean_encoded['subdistrict'])

# สร้าง X และ y
X = df_final_clean_encoded
y = df_final_clean['cases']

# แบ่งข้อมูลเป็นชุดฝึกและทดสอบ
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# สร้างโมเดล Gradient Boosting
gb_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=5)

# ฝึกโมเดล
gb_model.fit(X_train, y_train)

# ทำนายค่าบนชุดทดสอบ
y_pred = gb_model.predict(X_test)

# คำนวณค่า MSE (Mean Squared Error)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)  # คำนวณค่ารากที่สองของ MSE
print(f"RMSE: {rmse}")

# คำนวณค่า R² (Coefficient of Determination)
r2 = r2_score(y_test, y_pred)
print(f"R²: {r2}")

# กราฟเปรียบเทียบผลทำนายและค่าจริง
plt.figure(figsize=(10,6))
plt.plot(y_test.values, label='Actual Cases', color='red')
plt.plot(y_pred, label='Predicted Cases', color='blue')
plt.title('Predicted vs Actual Cases')
plt.xlabel('Index')
plt.ylabel('Number of Cases')
plt.legend()
plt.show()
